# Model training

#### 1. Prepare dataset:

In [84]:
# Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from scipy import stats
import joblib
import torch
from sklearn.preprocessing import PowerTransformer

In [85]:
# Declare the env variables
NAME_COMPANY = "processed_data_YesBank_StockPrices.csv"
PATH_PROCESS_INFO = f"../data/processed/{NAME_COMPANY}"
PATH_MODEL = f"../models/model_lstm.pkl"
BOX_COX_TRANSFORMER_PATH = "../config/model/transformers_boxcox.pkl"

In [86]:
# Import the data transform to start the training process
dataset_processed = pd.read_csv(PATH_PROCESS_INFO)

In [87]:
dataset_processed.head()

,Open,High,Low,Close
0,2.613965,2.682707,2.493722,2.568818
1,2.579869,2.745719,2.609925,2.645822
2,2.651627,2.745024,2.585922,2.636499
3,2.629821,2.716832,2.597132,2.612025
4,2.641559,2.673812,2.637552,2.645049


In [88]:
# Convertir el DataFrame a un NumPy array
data_numpy = dataset_processed.values
data_numpy

array([[2.61396481, 2.68270663, 2.49372188, 2.56881781],
       [2.57986883, 2.74571874, 2.60992489, 2.64582239],
       [2.65162713, 2.74502359, 2.58592243, 2.63649926],
       [2.62982127, 2.71683211, 2.59713216, 2.61202534],
       [2.64155903, 2.67381241, 2.63755175, 2.6450486 ],
       [2.65239763, 2.71468682, 2.64742699, 2.66801832],
       [2.6669312 , 2.89326531, 2.69393755, 2.78408128],
       [2.79685703, 2.8817327 , 2.82823193, 2.83636029],
       [2.84286095, 3.1002372 , 2.87046334, 3.06534175],
       [3.09160247, 3.09277568, 2.99657994, 3.03421148],
       [3.0522345 , 3.14154742, 2.8556634 , 2.95299616],
       [2.96424459, 2.96556888, 2.679714  , 2.80357439],
       [2.82991999, 2.89567715, 2.71261887, 2.8389397 ],
       [2.82338962, 2.97677804, 2.84886959, 2.94195337],
       [2.95271375, 2.9922725 , 2.92138415, 2.97757004],
       [2.99699127, 3.25012838, 3.02481144, 3.19717919],
       [3.19980204, 3.41934506, 3.18152504, 3.30783349],
       [3.3129648 , 3.46204029,

In [89]:
# Convertir el NumPy array a tensor
data_tensor = torch.tensor(data_numpy, dtype=torch.float32)
data_tensor

tensor([[2.6140, 2.6827, 2.4937, 2.5688],
        [2.5799, 2.7457, 2.6099, 2.6458],
        [2.6516, 2.7450, 2.5859, 2.6365],
        [2.6298, 2.7168, 2.5971, 2.6120],
        [2.6416, 2.6738, 2.6376, 2.6450],
        [2.6524, 2.7147, 2.6474, 2.6680],
        [2.6669, 2.8933, 2.6939, 2.7841],
        [2.7969, 2.8817, 2.8282, 2.8364],
        [2.8429, 3.1002, 2.8705, 3.0653],
        [3.0916, 3.0928, 2.9966, 3.0342],
        [3.0522, 3.1415, 2.8557, 2.9530],
        [2.9642, 2.9656, 2.6797, 2.8036],
        [2.8299, 2.8957, 2.7126, 2.8389],
        [2.8234, 2.9768, 2.8489, 2.9420],
        [2.9527, 2.9923, 2.9214, 2.9776],
        [2.9970, 3.2501, 3.0248, 3.1972],
        [3.1998, 3.4193, 3.1815, 3.3078],
        [3.3130, 3.4620, 3.3232, 3.3740],
        [3.3925, 3.6095, 3.4328, 3.4913],
        [3.4947, 3.5607, 3.4248, 3.4367],
        [3.4629, 3.5275, 3.2738, 3.4185],
        [3.3771, 3.5409, 3.4002, 3.4965],
        [3.5106, 3.6684, 3.5145, 3.6020],
        [3.6160, 3.6753, 3.5616, 3

#### 2. Define the model structure:

In [90]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Inicializar los estados ocultos y las celdas
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pasar la entrada por la LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Pasar la salida de la LSTM por la capa lineal
        out = self.fc(out[:, -1, :])  # Tomar la última salida de la secuencia
        return out



# Parámetros
input_size = 4   # Número de características de entrada
hidden_size = 50  # Tamaño de la capa oculta
num_layers = 2    # Número de capas LSTM
output_size = 1   # Número de características de salida (en este caso, el precio de cierre)

# Crear el modelo
model = LSTMModel(input_size, hidden_size, num_layers, output_size)


#### 3. Traning model Try 1: With 50 epochs:

In [91]:
import torch.optim as optim

# Configurar la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(data_tensor) * 0.8)
test_size = len(data_tensor) - train_size
train_data = data_tensor[:train_size]
test_data = data_tensor[train_size:]

# Crear DataLoader para los datos de entrenamiento y prueba
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# Entrenamiento
num_epochs = 50

for epoch in range(num_epochs):
    for batch in train_loader:
        # Los datos deben tener forma (batch_size, seq_len, input_size)
        batch = batch.unsqueeze(1)  # Añadir dimensión de secuencia
        outputs = model(batch)
        loss = criterion(outputs, batch[:, -1, :])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


/opt/anaconda3/envs/datapath-mlops-mike/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 4])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/envs/datapath-mlops-mike/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 4])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Loss: 16.7894
Epoch [2/50], Loss: 18.5285
Epoch [3/50], Loss: 19.9328
Epoch [4/50], Loss: 18.2754
Epoch [5/50], Loss: 16.1228
Epoch [6/50], Loss: 17.9610
Epoch [7/50], Loss: 14.8300
Epoch [8/50], Loss: 16.0894
Epoch [9/50], Loss: 12.5965
Epoch [10/50], Loss: 15.9026
Epoch [11/50], Loss: 15.2565
Epoch [12/50], Loss: 10.8868
Epoch [13/50], Loss: 11.1081
Epoch [14/50], Loss: 8.3635
Epoch [15/50], Loss: 9.0330
Epoch [16/50], Loss: 6.4391
Epoch [17/50], Loss: 3.9665
Epoch [18/50], Loss: 4.5360
Epoch [19/50], Loss: 2.8014
Epoch [20/50], Loss: 2.4265
Epoch [21/50], Loss: 1.8758
Epoch [22/50], Loss: 0.9108
Epoch [23/50], Loss: 0.7739
Epoch [24/50], Loss: 0.7933
Epoch [25/50], Loss: 0.7103
Epoch [26/50], Loss: 0.7648
Epoch [27/50], Loss: 0.3409
Epoch [28/50], Loss: 0.5853
Epoch [29/50], Loss: 0.6043
Epoch [30/50], Loss: 0.6267
Epoch [31/50], Loss: 0.4456
Epoch [32/50], Loss: 0.5171
Epoch [33/50], Loss: 0.7421
Epoch [34/50], Loss: 0.4340
Epoch [35/50], Loss: 0.4713
Epoch [36/50], L

### 3.1. Evaluate model after train with 50 epochs

In [92]:
# Evaluación
model.eval()
with torch.no_grad():
    predictions = []
    actuals = []
    
    for batch in test_loader:
        batch = batch.unsqueeze(1)
        outputs = model(batch)
        predictions.append(outputs)
        actuals.append(batch[:, -1, :])
    
    # Convertir las listas a tensores
    predictions = torch.cat(predictions, dim=0)
    actuals = torch.cat(actuals, dim=0)
    
    # Calcular métricas de evaluación
    mse = criterion(predictions, actuals)
    print(f'Test MSE: {mse.item():.4f}')


Test MSE: 0.9480


/opt/anaconda3/envs/datapath-mlops-mike/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([37, 4])) that is different to the input size (torch.Size([37, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


#### 4. Training model Try 2: 100 epochs

In [93]:
# Entrenamiento
num_epochs = 100

for epoch in range(num_epochs):
    for batch in train_loader:
        # Los datos deben tener forma (batch_size, seq_len, input_size)
        batch = batch.unsqueeze(1)  # Añadir dimensión de secuencia
        outputs = model(batch)
        loss = criterion(outputs, batch[:, -1, :])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 0.3672
Epoch [2/100], Loss: 0.6559
Epoch [3/100], Loss: 0.6063
Epoch [4/100], Loss: 0.3469
Epoch [5/100], Loss: 0.3492
Epoch [6/100], Loss: 0.4440
Epoch [7/100], Loss: 0.3522
Epoch [8/100], Loss: 0.2509
Epoch [9/100], Loss: 0.2384
Epoch [10/100], Loss: 0.3018
Epoch [11/100], Loss: 0.2348
Epoch [12/100], Loss: 0.4031
Epoch [13/100], Loss: 0.2122
Epoch [14/100], Loss: 0.3108
Epoch [15/100], Loss: 0.1669
Epoch [16/100], Loss: 0.1702
Epoch [17/100], Loss: 0.2322
Epoch [18/100], Loss: 0.1403
Epoch [19/100], Loss: 0.3811
Epoch [20/100], Loss: 0.3165
Epoch [21/100], Loss: 0.2490
Epoch [22/100], Loss: 0.2771
Epoch [23/100], Loss: 0.1002
Epoch [24/100], Loss: 0.1751
Epoch [25/100], Loss: 0.1883
Epoch [26/100], Loss: 0.1206
Epoch [27/100], Loss: 0.1487
Epoch [28/100], Loss: 0.1255
Epoch [29/100], Loss: 0.1321
Epoch [30/100], Loss: 0.1306
Epoch [31/100], Loss: 0.0636
Epoch [32/100], Loss: 0.1309
Epoch [33/100], Loss: 0.1176
Epoch [34/100], Loss: 0.1019
Epoch [35/100], Loss: 0

### 4.2. Evaluate model after train with 50 epochs

In [94]:
# Evaluación
model.eval()
with torch.no_grad():
    predictions = []
    actuals = []
    
    for batch in test_loader:
        batch = batch.unsqueeze(1)
        outputs = model(batch)
        predictions.append(outputs)
        actuals.append(batch[:, -1, :])
    
    # Convertir las listas a tensores
    predictions = torch.cat(predictions, dim=0)
    actuals = torch.cat(actuals, dim=0)
    
    # Calcular métricas de evaluación
    mse = criterion(predictions, actuals)
    print(f'Test MSE: {mse.item():.4f}')

Test MSE: 0.1156


In [95]:
# Save model
joblib.dump(model, PATH_MODEL)

['../models/model_lstm.pkl']

### 5. Dataset de prueba para obtener una nueva predicción

In [96]:
# Cargar el transformador guardado (para uso futuro)
loaded_transformer = joblib.load(BOX_COX_TRANSFORMER_PATH)

# Nuevos datos (debe tener el mismo número de características que el transformador espera)
new_info = pd.DataFrame({
    'Open': [12, 17.16, 11.85, 14.37],
    'High': [14.3, 15.34, 12.75, 13.15],
    'Low': [13.3, 14.01, 12.11, 12.42],
    'Close': [12.41, 14.9, 12.21, 14.67]
})

# Aplicar la transformación a los nuevos datos
new_info_transformed = pd.DataFrame(loaded_transformer.transform(new_info))

print("Nuevos datos transformados:")
print(new_info_transformed)


Nuevos datos transformados:
          0         1         2         3
0  2.530892  2.704617  2.671731  2.564649
1  2.902864  2.777207  2.727185  2.754498
2  2.517846  2.586126  2.571966  2.547804
3  2.718088  2.618013  2.598847  2.738327


In [97]:
# Convertimos en array tensor el dataset de
new_info_transformed_numpy = np.array(new_info_transformed)
new_info_transformed_numpy_tensor = torch.tensor(new_info_transformed_numpy, dtype=torch.float32)

In [98]:
new_info_transformed_numpy_tensor

tensor([[2.5309, 2.7046, 2.6717, 2.5646],
        [2.9029, 2.7772, 2.7272, 2.7545],
        [2.5178, 2.5861, 2.5720, 2.5478],
        [2.7181, 2.6180, 2.5988, 2.7383]])

In [99]:
# Tensor proporcionado
#tensor = torch.tensor([
#    [2.5309, 2.7046, 2.6717, 2.5646],
#    [2.9029, 2.7772, 2.7272, 2.7545],
#    [2.5178, 2.5861, 2.5720, 2.5478],
#    [2.7181, 2.6180, 2.5988, 2.7383]
#], dtype=torch.float32)

# Añadir dimensión de batch_size
# tensor = tensor.unsqueeze(0)  # Forma final: [1, 4, 4]
tensor = new_info_transformed_numpy_tensor.unsqueeze(0)

# Verificar la forma del tensor
print(f'Forma del tensor de entrada: {tensor.shape}')

# Convertir el tensor a DataFrame si lo necesitas
df = pd.DataFrame(tensor.squeeze(0).numpy(), columns=['Feature1', 'Feature2', 'Feature3', 'Feature4'])
print(df)

Forma del tensor de entrada: torch.Size([1, 4, 4])
   Feature1  Feature2  Feature3  Feature4
0  2.530892  2.704617  2.671731  2.564649
1  2.902864  2.777207  2.727185  2.754498
2  2.517847  2.586126  2.571966  2.547804
3  2.718088  2.618013  2.598846  2.738327


In [100]:
# Cargar el modelo guardado
model_cargado = joblib.load('../models/model_lstm.pkl')


# Asegurarse de que el modelo esté en modo evaluación
model_cargado.eval()

# Tensor de entrada (ya ajustado a [batch_size, sequence_length, input_size])
# Realizar la predicción
with torch.no_grad():
    #predicciones = model(tensor)
    predicciones = model_cargado(tensor)

# Convertir las predicciones a NumPy para una visualización más fácil
predicciones_numpy = predicciones.numpy()

# Mostrar las predicciones
print("Predicciones:")
print(predicciones_numpy)


Predicciones:
[[5.6601396]]
